# Experiments with RSUE (Istanbul and Aarhus)

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Prepare the database

In [4]:
import psycopg2 as pg2
con = {'host':'127.0.0.1','port':5432,'db':'osm_po','user':'postgres'}


In [101]:
cd ~/projects/Istanbul_2015/

/Users/alehmann/projects/Istanbul_2015


In [102]:
cities = {
          'AAR':'https://s3.amazonaws.com/metro-extracts.mapzen.com/aarhus_denmark.osm.pbf',
          'IST': 'https://s3.amazonaws.com/metro-extracts.mapzen.com/istanbul_turkey.osm.pbf'
}

def osm2csv(table):
    !psql -U postgres postgres://$host:$port/osm_po -o $table -c "SELECT * FROM $table;"

#osm2csv('ist_2po_4pgr')            


for city in cities:
#Get OSM data
    url = cities[city]
    print url
    !wget $url -P ~/Downloads
    path = '/Users/alehmann/Downloads/'+url.split('/')[-1]
    res = !java -jar osm2po-core-5.0.0-signed.jar cmd=tjsp prefix=$city $path
    sql_file = res[-2].split()[-1]
    
    !psql -U postgres postgres://$host:$port/osm_po -f $sql_file
    #psql -Upostgres -l postgres://46.101.171.24:6432/osm_po 
    #-o ist_1306.csv -c "SELECT * FROM ist13062015_2po_4gr"
    !rm $path

https://s3.amazonaws.com/metro-extracts.mapzen.com/istanbul_turkey.osm.pbf
--2015-06-29 13:46:27--  https://s3.amazonaws.com/metro-extracts.mapzen.com/istanbul_turkey.osm.pbf
Resolving s3.amazonaws.com... 54.231.8.160
Connecting to s3.amazonaws.com|54.231.8.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8871418 (8.5M) [binary/octet-stream]
Saving to: '/Users/alehmann/Downloads/istanbul_turkey.osm.pbf'

istanbul_turkey.osm 100%[=====================>]   8.46M  1.29MB/s   in 7.5s   

2015-06-29 13:46:35 (1.13 MB/s) - '/Users/alehmann/Downloads/istanbul_turkey.osm.pbf' saved [8871418/8871418]

psql: option requires an argument -- f
Try "psql --help" for more information.
https://s3.amazonaws.com/metro-extracts.mapzen.com/aarhus_denmark.osm.pbf
--2015-06-29 13:46:43--  https://s3.amazonaws.com/metro-extracts.mapzen.com/aarhus_denmark.osm.pbf
Resolving s3.amazonaws.com... 54.231.8.160
Connecting to s3.amazonaws.com|54.231.8.160|:443... connected.
HTTP request 

In [3]:

def get(db, query):
    cur = db.cursor()
    cur.execute(query)
    return cur.fetchall()

def set1(db, query,data):
    cur = db.cursor()
    cur.execute(query,data)
    return cur.fetchall()

def setM(db, query,data):
    cur = db.cursor()
    cur.executemany(query,data)
    return cur.fetchall()


In [2]:
temp_table ='ist_temp'

def drop_tables(cur,temp_table):
    cur.execute("DROP TABLE path2link;")
    cur.execute("DROP TABLE od_matrix;")
    cur.execute("DROP TABLE od_path;")
    cur.execute("DROP TABLE {};".format(temp_table))

def create_tables(db,table,temp_table):
    # Create a copy of the main edge table
    copy_table = "DROP TABLE IF EXISTS {0} CASCADE;\
        CREATE TABLE {0} as SELECT * from {1};".format(temp_table,table)
    # Add capacity,flow,flow dependent cost primary key and indeces
    capacity = "ALTER table {} add column capacity integer DEFAULT 2800;\
                ".format(temp_table)
    flow = "ALTER table {} add column flow double precision DEFAULT 0;".format(temp_table)
    flowcost = "ALTER table {0} add column dyncost double precision ;\
                update {0} set dyncost = cost;\
                ".format(temp_table)
    pkey = "alter table {0} add constraint pkey_ist_temp PRIMARY KEY (id);\
            CREATE INDEX idx_{0}_source ON {0}\
              USING btree (source);\
            CREATE INDEX idx_{0}_target ON {0}\
              USING btree (target);".format(temp_table)
    # The table that contains the considered OD paths
    # For each OD pair multiple path can occur as congestion manifests itself
    # the (transformed) cost and the flow is update during the algorithm
    od_table = "DROP TABLE IF EXISTS od_matrix CASCADE;\
                CREATE TABLE od_matrix (id serial PRIMARY key, \
                        origin integer,\
                        destination integer,\
                        demand integer);".format(temp_table)

    path_table = "DROP TABLE IF EXISTS od_path CASCADE;\
                    CREATE TABLE od_path (id serial PRIMARY key,\
                    od integer references od_matrix (id),\
                    hash bytea UNIQUE,\
                    cost float, flow float );"
    # The link for many to many relationships.
    # Each link can occur in multiple paths and each path contains multiple links
    path2link = """DROP TABLE IF EXISTS path2link CASCADE;
                    CREATE TABLE path2link (
                    pathid integer references od_path (id), 
                    link_id integer references {} (id));
                   """.format(temp_table)
    constants = "drop table if exists constants;\
                    create table constants ( alfa float, \
                    beta float DEFAULT 4, theta float, beta_ps float);\
                    insert into constants values (0.5,4,0.1,-8.5);"

    sql_create = [copy_table,capacity,flow,flowcost,pkey,od_table,
                  path_table,path2link,constants]
    if 1:
        command = '\n'.join(sql_create)
#    for command in sql_create:
        cur = db.cursor()
        print command
        cur.execute(command)
        db.commit()


def od_matrix(db,data):
    # The Origin Destination table. A static table with the demand data
    
    insert = "INSERT INTO od_matrix (origin,destination,demand) VALUES (%s,%s,%s)"
    cur = db.cursor()
    cur.executemany(insert,data)
    db.commit()

def od_path(db):
    populate = "INSERT INTO od_path (source,destination,cost,flow) VALUES (%s,%s,%s,%s)"
    
    cur = db.cursor()
    cur.execute("SELECT origin,destination,0,demand FROM od_matrix;")
    vals = cur.fetchall()
    print vals
    cur.executemany(populate,vals)
    
def getODpairs(db):
    cur = db.cursor()
    cur.execute("select origin,array_agg(id),array_agg(destination),\
                    array_agg(demand) from od_matrix group by origin;")
    res = cur.fetchall()
    return res

def orderPath(db):
    cur = db.cursor()
    cur.execute("select * from path2link order by costXXX")
    return cur.fetchall()

    find_path_end = "drop table if exists endpoint; \
                    create table endpoint \
                    (id serial, begin integer,endpoint integer);\
                    insert into endpoint(begin,endpoint) \
                    select 0,seq from temp where id3=-1 limit 1;\
                    drop table if exists temp_point;\
                    create table temp_point\
                    (id serial, begin integer,endpoint integer);\
                    insert into temp_point (begin,endpoint) \
                    select 0,seq from temp where id3=-1;\
                    insert into endpoint (begin,endpoint)\
                    Select t1.endpoint+1,t2.endpoint from temp_point t1 \
                    join temp_point t2 on t2.id =t1.id+1;\
                    drop table if exists cost_of_new_path;\
                    create table cost_of_new_path as\
                    select {} as source,id1 as target,sum(cost) as cost from temp inner join\
                    endpoint as end1 on \
                    temp.seq>end1.begin and temp.seq<end1.endpoint \
                    group by id1;".format(source)



In [5]:
db = pg2.connect('postgres://{user}@{host}:{port}/{db}'.format(**con))
%time create_tables(db,'ist_2po_4pgr','ist_temp')
data2 = [(173656, 187633, 400),
 (173656, 132750, 900),
 (173656, 50429, 810),
 (173656, 4783, 220),
 (173656, 2281, 230),
 (173656, 2761, 240),
 (187633,173656,1850),
 (187633,2761,1860)]
od_matrix(db,data2)

db.commit()
ods = getODpairs(db)
print ods

NameError: name 'pg2' is not defined

In [362]:
import time

def get_K_routes(db,table,source,dest,demand,direct=False,reverse=False):
    pg_kdijkstra = ("""pgr_kdijkstrapath('SELECT id, source,
                target, dyncost as cost, x1, y1, x2, y2 
               from {table}',{source},array{dest},{direct},{reverse})""".
                    format(table=table, source=source,
                           dest=dest,direct=direct,reverse=reverse))
    route = """DROP TABLE IF EXISTS temp;
            CREATE TABLE temp AS SELECT {} as source,* \
            FROM {}""".format(source,pg_kdijkstra)
    
    path_cost_dijk = "drop table if exists new_paths_costs;\
                        create table new_paths_costs select {} as source,id1 as target,sum(cost) as cost \
                        from temp group by id1;".format(source)
                
    od_path_min = "drop table if exists min_od_costs;\
                    create table min_od_costs as select od,min(cost) from od_path group by od;"
    
    md5="drop table if exists md5oflinks; \
            create table md5oflinks as \
            select id1,digest((string_agg(\
            to_char(id3,'999999'),'')),'md5') as hash\
            from temp group by id1;"

    #get_viable_path = "select od from (select id as od, cost from new_paths_costs join od_matrix on\
    #                origin=source and destination=target), min_od_costs where cost<min_od_costs.cost;"
    
    get_path_if_cheaper"drop table if exists cheaper_path;\
                            create table cheaper_path as select newp.od,newp.origin,\
                            newp.destination,0 as cost,newp.demand,min_od_costs.min from \
                            (select id as od, cost,origin,destination,demand from \
                            new_paths_costs join od_matrix on origin=source and destination=target)\
                            as newp join min_od_costs on newp.od=min_od_costs.od\
                            where newp.cost<=min_od_costs.min;"
    
    odpairs = "DROP TABLE IF EXISTS odpairs_for_source;\
            create table odpairs_for_source as \
            select od_matrix.id as od,origin,destination,\
            0 as cost,demand from od_matrix \
            where od_matrix.origin={};"
    
    insert_path="INSERT INTO od_path (od,hash,cost,flow) \
                select od,hash,cost,0 from \
                md5oflinks,odpairs_for_source where id1=destination;" 
    
    get_min_cost = "drop table if exists min_od_cost;\
                    create table min_od_cost as \
                    select od,min(cost) from od_path group by od;"
    start = time.time()
    cur = db.cursor()
    cur.execute(get_min_cost)
    cur.execute(route)
    db.commit()
    cur = db.cursor()
    cur.execute(find_path_end)
    db.commit()
    cur = db.cursor()
    cur.execute(md5)
    cur.execute(odpairs.format(source))
    cur.execute(insert_path)
    db.commit()
    cur = db.cursor()
    end = time.time()
    print "Dijkstra:",end-start
    start = end
    
    p2l ="select od_path.id,temp.id3 from\
            temp, od_path inner join od_matrix\
            on od_path.od=od_matrix.id where \
            temp.id3>-1 and temp.id1=od_matrix.destination \
            EXCEPT select pathid,link_id from path2link;"  
    insert="INSERT INTO path2link (pathid,link_id) \
                    {}".format(p2l)
                                   

    cur.execute(insert)
    end = time.time()
    print "Insert into 'path2link: {}".format(end-start)
    start = end        

#db = pg2.connect('postgres://{user}@{host}:{port}/{db}'.format(**con))
#for od in ods:
#    print od
#    get_K_routes(db,'ist_temp',od[0],od[2],od[3])
#db.commit()

In [246]:
def initial_loading(db):
    cur = db.cursor()
    cur.execute("update od_path set flow = demand\
                from od_matrix where od_matrix.id = od_path.od;")
    db.commit()
initial_loading(db)    

The Path Size logit is describing the probability of choosing a path by the equation:

$ Pr_{mr}(\mathbf{c}(\mathbf{x})|R_{m}) = \frac{exp(-\theta c_{mr}(\mathbf{x})+\beta_{PS}ln(PS_{mr}))}{\sum_{s\in R_m}{exp(-\theta c_{ms}(\mathbf{x})+\beta_{PS}ln(PS_{ms}))}}$
where $PS_{mr}$ is given by:

$PS_{mr}=\sum_{a\in\gamma}\frac{l_a}{L_{mr}}\frac{1}{\sum_{k\in R_m}{\delta_{amk}}}$

Here we notice that $L_{mr}$ does not depend on $a$. We can then move it outside the sum:

$PS_{mr}=\frac{1}{L_{mr}}\sum_{a\in\gamma}\frac{l_a}{\sum_{k\in R_m}{\delta_{amk}}}$


To calculate the PS logit, we first calculate the PS by the following SQL, which will calculate the sum:

```SQL
SELECT SUM(x.y) FROM
    (SELECT l.cost/d.delta as y FROM ist_temp l 
    INNER JOIN 
      (SELECT link_id, COUNT(link_id) AS delta FROM path2link GROUP BY link_id) as d 
    ON l.id=d.link_id) as x;```


Then we calculate $L_{mr}$ by creating a table to hold the results:

```SQL
create table Lmr as 
select pathid,sum(cost) as Lmr from path2link 
group by pathid;on p.link_id; ```

For use in the RSUE solution we need to calculate the transformed cost :

$\tilde{c}(\mathbf{x})= \chi_{mr}exp(\theta(c_{mr}(\mathbf{x})+\beta_{PS}ln(PS_{mr}))) $

We can calculate $PS_{mr}$ by combining the results of the above two SQL statements and we can calculate the current cost $c_{mr}(\mathbf{x})$ by the following SQL:

```SQL

SELECT pathid,sum(cost*(1+0.5*(flow/capacity)^4)) from path2link group by pathid;```

The transformed cost os also used in the relative gap calculation (step 4), which is used as a convergence measure, and in the pairwise swap algorithm (step 2)


In [258]:
def generalised_cost(db):
    # Update temporary edge table with total flow on every used link
    # Done in three steps:
    #    Create a temporary table containing the new flow on all used links
    #    Update ist_temp flow from the temporary table
    #    update ist_temp dyncost with BPR calculation from the flow
    
    flow_from_path = "drop table if exists temp2;\
                    create table temp2 as select link_id,sum(i.flow) as flow \
                        FROM path2link \
                        JOIN od_path i ON\
                        i.id = pathid group by link_id;"
    update_flow =   "update ist_temp set flow= t.flow \
                        from temp2 t where t.link_id = id;"
    update_dyncost = "update ist_temp as i set dyncost = \
                        i.cost*(1+0.5*(t.flow/i.capacity)^4) from \
                        temp2 t where t.link_id = i.id;"
 
    cost_path = "drop table if exists pathsum;\
                create table pathsum as \
                select pathid,sum(i.cost) as cost \
                from path2link JOIN ist_temp i ON\
                i.id= link_id group by pathid;\
                update od_path set cost=pathsum.cost from pathsum\
                where od_path.id=pathsum.pathid"#Cost of path
    cur = db.cursor()    
    cur.execute(flow_from_path)
    cur.execute(update_flow)
    cur.execute(update_dyncost)
    cur.execute(cost_path)
    db.commit()
    
def transformed_cost(db):
    t_cost="drop table if exists trans_cost;\
            create table trans_cost as\
            select od_path.id,od_path.od,\
            exp(constants.theta*(od_path.cost+\
            constants.beta*ln(ps_path.ps)))*od_path.flow as tcost from\
            od_path join ps_path on od_path.id=ps_path.pathid\
            join constants on 1=1;"
    cur = db.cursor()
    cur.execute(t_cost)
    db.commit()
    

db = pg2.connect('postgres://{user}@{host}:{port}/{db}'.format(**con))
generalised_cost(db)    
db.commit()

In [259]:
Lmr = "drop table if exists lmr; \
       create table Lmr as select pathid,sum(i.cost) as Lmr \
            from path2link JOIN ist_temp i\
            ON i.id=link_id group by pathid;"
    
#To get the sum of deltas (denominator of eq 24)
la_over_deltasum = "drop table if exists link_ps;\
                    create table link_ps as \
                    select d.link_id,l.dyncost/d.delta as ps from ist_temp l\
                    inner join \
                    (select link_id,count(link_id) as delta \
                    from path2link group by link_id) as d \
                    on l.id=d.link_id;"
ps ="drop table if exists PS;\
     create table PS as Select p2l.pathid,sum(lps.ps) as sum \
                     from path2link p2l\
                     JOIN link_ps lps ON lps.link_id=p2l.link_id\
                     group by p2l.pathid; "
ps_total="drop table if exists ps_path;\
            create table ps_path as \
            select lmr.pathid,ps.sum/lmr.lmr as ps from ps join lmr \
            on lmr.pathid = ps.pathid;"
def PS(db):
    # Save the Lmr in a table
    # The Lmr is the sum of cost of the links the path consists of
    
    cur = db.cursor()
    cur.execute(Lmr)
    cur.execute(la_over_deltasum)
    cur.execute(ps)
    cur.execute(ps_total)


db = pg2.connect('postgres://{user}@{host}:{port}/{db}'.format(**con))
PS(db)
db.commit()
db = pg2.connect('postgres://{user}@{host}:{port}/{db}'.format(**con))
transformed_cost(db)
db.commit()

In [345]:
class SortnSwap(object):

    def __init__(self,d=0):
        self.n = 0
        self.d = d
    def sort_and_swap(self,db):

        sort_odpath = "select id,od,cost,flow from od_path order by od,cost DESC;"
        cur = db.cursor()
        self.n += 1
        denom = sum([x**(self.d) for x in range(self.n+1)])
        gamma = ((1.0*self.n)**self.d)/denom
        print 'SortnSwap',self.n,denom,gamma
        cur.execute(sort_odpath)
        res = cur.fetchall()
        od_path = {}
        for row in res:
            od_path.setdefault(row[1],[]).append((row[0],row[2],row[3]))
        res = []
        for k in od_path:
            l = od_path[k]
            pair = zip(l,reversed(l))[:len(l)/2]
            for hi,lo in pair:
                Gamma = (hi[1]-lo[1])/sqrt(hi[1]**2+lo[1]**2)
                lo_f = lo[2]+Gamma*hi[2]*gamma
                hi_f = hi[2]-Gamma*hi[2]*gamma
                res.append((hi[0],hi_f))
                res.append((lo[0],lo_f))
                update="update od_path set flow={} where id ={};"
                cur.execute(update.format(lo_f,lo[0]))
                cur.execute(update.format(hi_f,hi[0]))

        db.commit()
s = SortnSwap()
s.sort_and_swap(db)

SortnSwap 1 2 0.5


InternalError: current transaction is aborted, commands ignored until end of transaction block


In [343]:
# Step 0
# Get O-D Matrix
# Create solution table (id, O,D, cost, ...
# for every O :
#     Use kDikstrapath to get shortest path to all destintions 
# Do all or nothing assignment
# Calculate flow and generalised cost
def step0(db,OD_data):
    
    create_tables(db,'ist_2po_4pgr','ist_temp')
    od_matrix(db,OD_data)
    ods = getODpairs(db)
    db.commit()
    for od in ods:
        get_K_routes(db,'ist_temp',od[0],od[2],od[3]) 
        db.commit()
    insert_into_od_path(db)
    cur = db.cursor()
    cur.execute("update od_path set flow = demand\
                from od_matrix where od_matrix.id = od_path.od;")
    db.commit()
    insert_into_path2links(db)
    generalised_cost(db) 
    PS(db)
    transformed_cost(db)

    db.commit()


        
# Step 1
    
# for every O :
#     Use kDikstra to get shortest path to all destinations 
#          get shortest path with current cost and flow
# If new shortest path are found add to solution table with zero flow

def step1(db):
    for od in ods:
        get_K_routes(db,'ist_temp',od[0],od[2],od[3]) 
    
# Step 2
# for each path in table calculate the tranformed cost
#    calculate the flows
#    calculate the generalised cost
#    calculate the transformed cost (wirh PS-logit
# Sort and pair and swap
def step2(db):
    generalised_cost(db) 
    PS(db)
    transformed_cost(db)

    s.sort_and_swap(db)

# step 3
# Network loading and generalised cost calculation
def step3(db):
    generalised_cost(db) 
    PS(db)
    transformed_cost(db)

# Step 4
# Gap calculation and convergence check
def step4(db):
    denom ="select sum(tcost*flow) \
            from trans_cost join od_path on \
            trans_cost.id=od_path.id;"
    min_tcost_table = "drop table if exists tcost_vs_min;\
                create table tcost_vs_min as \
                select id,t.od,t.tcost,m.tcost as mcost from \
                trans_cost t join (select od,min(tcost) tcost from\
                trans_cost group by od) as m on \
                t.od=m.od order by t.od;"
    numerator = "Select sum(flow*(tcost-mcost)) from tcost_vs_min join\
                od_path on od_path.id = tcost_vs_min.id;"
    cur = db.cursor()
    cur.execute(denom)
    de = cur.fetchone()[0]
    cur.execute(min_tcost_table)
    cur.execute(numerator)
    mini = cur.fetchone()[0]
    return mini/de

In [344]:
step4(db)

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [363]:
%%time
db = pg2.connect('postgres://{user}@{host}:{port}/{db}'.format(**con))
s = SortnSwap(2)
step0(db,data2)


DROP TABLE IF EXISTS ist_temp CASCADE;        CREATE TABLE ist_temp as SELECT * from ist_2po_4pgr;
ALTER table ist_temp add column capacity integer DEFAULT 2800;                
ALTER table ist_temp add column flow double precision DEFAULT 0;
ALTER table ist_temp add column dyncost double precision ;                update ist_temp set dyncost = cost;                
alter table ist_temp add constraint pkey_ist_temp PRIMARY KEY (id);            CREATE INDEX idx_ist_temp_source ON ist_temp              USING btree (source);            CREATE INDEX idx_ist_temp_target ON ist_temp              USING btree (target);
DROP TABLE IF EXISTS od_matrix CASCADE;                CREATE TABLE od_matrix (id serial PRIMARY key,                         origin integer,                        destination integer,                        demand integer);
DROP TABLE IF EXISTS od_path CASCADE;                    CREATE TABLE od_path (id serial PRIMARY key,                    od integer references od_matrix (i

In [364]:
%%time
for i in range(10):
    start = time.time()
    db.commit()
    step1(db)
    step2(db)
    step3(db)
    print '========>',step4(db)
    end = time.time()
    print "Loop time {}".format(end-start)
    db.commit()

Dijkstra: 0.438166856766
Insert into 'path2link: 0.033135175705
Dijkstra: 0.440097093582
Insert into 'path2link: 0.0115678310394
SortnSwap 1 1 1.0
========> 0.228718985266
Loop time 1.07918906212
Dijkstra: 0.417949914932
Insert into 'path2link: 0.0382912158966
Dijkstra: 0.414095878601
Insert into 'path2link: 0.0118281841278
SortnSwap 2 5 0.8
========> 0.298293962517
Loop time 1.04698395729
Dijkstra: 0.406997919083
Insert into 'path2link: 0.0838952064514
Dijkstra: 0.434854984283
Insert into 'path2link: 0.0527169704437
SortnSwap 3 14 0.642857142857
========> 0.575997180883
Loop time 1.8241288662
Dijkstra: 0.456661939621
Insert into 'path2link: 0.109045982361
Dijkstra: 0.42314696312
Insert into 'path2link: 0.0324430465698
SortnSwap 4 30 0.533333333333
========> 0.753899178598
Loop time 1.90392398834
Dijkstra: 0.451704025269
Insert into 'path2link: 0.068578004837
Dijkstra: 0.446114063263
Insert into 'path2link: 0.0723099708557
SortnSwap 5 55 0.454545454545
========> 0.832086767527
Loop tim

In [272]:
db.commit()
step2(db)


In [273]:
db.commit()
step3(db)

In [240]:
db.commit()

In [4]:
cur = db.cursor()

NameError: name 'db' is not defined

In [15]:
l = """md5oflinks_4401651712
 md5oflinks_4405858304
 md5oflinks_4410064896
 md5oflinks_4414271488
 md5oflinks_4418478080
 md5oflinks_4422684672
 md5oflinks_4426891264
 md5oflinks_4431097856""".split()
for t in l:
    !psql -U alehmann postgres://localhost:5432/osm_po -c "drop table $t"

DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE


In [13]:
l= """
50459

 """.split()
for pid in l:
    !psql -U alehmann postgres://localhost:5432/osm_po -c "select pg_cancel_backend($pid);"
    !psql -U alehmann postgres://localhost:5432/osm_po -c "select pg_terminate_backend($pid);"

 pg_cancel_backend 
-------------------
 t
(1 row)

 pg_terminate_backend 
----------------------
 t
(1 row)



In [5]:
from scipy.spatial.distance import pdist, squareform

In [6]:
from scipy.sparse import csr_matrix

In [7]:
import psycopg2

In [8]:
db = psycopg2.connect('postgres://{user}@{host}:{port}/{db}'.format(**con))

In [9]:
cur = db.cursor()
cur.execute("select max(source) from ist_temp")
res = cur.fetchone()[0]
mat = ndarray((res,res))
cur.execute("select source,target,cost from ist_temp")
while res:
    res = cur.fetchmany(10000)
    for r,c,cost in res:
        mat[r-1,c-1]=cost

In [1]:
from scipy.sparse.csgraph import dijkstra

In [ ]:
%timeit dijkstra(mat,indices =[1])

In [11]:
mat.to

(203488, 203488)

In [14]:
from cffi import FFI

In [15]:
ffi = FFI()

ImportError: dlopen(/usr/local/lib/python2.7/site-packages/_cffi_backend.so, 2): Library not loaded: /usr/local/opt/libffi/lib/libffi.6.dylib
  Referenced from: /usr/local/lib/python2.7/site-packages/_cffi_backend.so
  Reason: image not found